In [13]:
import json
import os
from pathlib import Path
import sys

# Add the current directory to Python path so we can import our modules
sys.path.append('.')

# Import our custom modules
from pdf_visual_extract import run_pdf_visual_extraction


# PDF Visual Extraction Demo

This notebook shows how to use the PDF Visual Extraction library to extract text, tables, and figures from PDFs.


In [ ]:
# Step 1: Run the extraction pipeline with concurrent processing
pdf_path = "../data/2025q1-alphabet-earnings-release.pdf"
output_dir = "../demo_output/2025q1-alphabet-earnings-release"

print("🚀 Running PDF Visual Extraction with concurrent VLM processing...")
print("⚡ Using 5 concurrent workers for faster processing")

# Run with concurrent processing (5 workers by default)
success = run_pdf_visual_extraction(
    pdf_path, 
    output_dir, 
    max_pages=10,
    export_md=True,
    force=False,
    max_workers=5  # 5 concurrent workers for VLM processing
)

if success:
    print("✅ Extraction completed successfully!")
else:
    print("❌ Extraction failed!")


🚀 Running PDF Visual Extraction...
🚀 Starting PDF Visual Extraction Pipeline
📄 PDF: ../data/2025q1-alphabet-earnings-release.pdf
📁 Output: ../demo_output/2025q1-alphabet-earnings-release/2025q1-alphabet-earnings-release
📊 Max pages: 10

STEP: PDF Text Extraction
Extracting text from: ../data/2025q1-alphabet-earnings-release.pdf
✅ Text extraction completed!
   Total pages: 10
   Saved to: ../demo_output/2025q1-alphabet-earnings-release/2025q1-alphabet-earnings-release/text_extraction/2025q1-alphabet-earnings-release_text.json

STEP: PDF to Images Conversion
Converting PDF to images...
Converting PDF to images with DPI: 300
Limiting to first 10 pages
Saved page 1: ../demo_output/2025q1-alphabet-earnings-release/2025q1-alphabet-earnings-release/images/page_000.png
Saved page 2: ../demo_output/2025q1-alphabet-earnings-release/2025q1-alphabet-earnings-release/images/page_001.png
Saved page 3: ../demo_output/2025q1-alphabet-earnings-release/2025q1-alphabet-earnings-release/images/page_002.pn

In [15]:
# Step 2: Load the extracted JSON data
json_file = os.path.join(output_dir, "text_extraction", "2025q1-alphabet-earnings-release_with_tables.json")

print(f"📄 Loading data from: {json_file}")

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"✅ Loaded data for: {data['pdf_name']}")
print(f"📊 Total pages: {data['total_pages']}")
print(f"📁 PDF path: {data['pdf_path']}")


📄 Loading data from: ../demo_output/2025q1-alphabet-earnings-release/text_extraction/2025q1-alphabet-earnings-release_with_tables.json
✅ Loaded data for: 2025q1-alphabet-earnings-release
📊 Total pages: 10
📁 PDF path: data/2025q1-alphabet-earnings-release.pdf


In [16]:
# Step 3: Explore the extracted data
print("🔍 Exploring extracted data...\n")

total_tables = 0
total_figures = 0

for page in data['pages']:
    page_num = page['page_number']
    text_length = len(page['text'])
    tables = page.get('tables', [])
    
    print(f"📄 Page {page_num}:")
    print(f"   Text: {text_length:,} characters")
    print(f"   Tables/Figures: {len(tables)}")
    
    for i, table in enumerate(tables, 1):
        element_type = table.get('type', 'table')
        description = table.get('description', 'No description')
        confidence = table.get('confidence', 'N/A')
        
        print(f"   {element_type.title()} {i}: {description[:50]}... (confidence: {confidence})")
        
        if element_type == 'table':
            total_tables += 1
        elif element_type == 'figure':
            total_figures += 1
    
    print()

print(f"📊 Summary:")
print(f"   Total Tables: {total_tables}")
print(f"   Total Figures: {total_figures}")
print(f"   Total Elements: {total_tables + total_figures}")


🔍 Exploring extracted data...

📄 Page 1:
   Text: 2,659 characters
   Tables/Figures: 0

📄 Page 2:
   Text: 2,717 characters
   Tables/Figures: 0

📄 Page 3:
   Text: 4,938 characters
   Tables/Figures: 0

📄 Page 4:
   Text: 1,962 characters
   Tables/Figures: 0

📄 Page 5:
   Text: 823 characters
   Tables/Figures: 0

📄 Page 6:
   Text: 1,941 characters
   Tables/Figures: 0

📄 Page 7:
   Text: 2,177 characters
   Tables/Figures: 0

📄 Page 8:
   Text: 2,212 characters
   Tables/Figures: 0

📄 Page 9:
   Text: 3,155 characters
   Tables/Figures: 0

📄 Page 10:
   Text: 534 characters
   Tables/Figures: 0

📊 Summary:
   Total Tables: 0
   Total Figures: 0
   Total Elements: 0


In [17]:
# Step 4: Show sample table data
print("📋 Sample Table Data:\n")

# Find the first table with structured data
sample_table = None
for page in data['pages']:
    for table in page.get('tables', []):
        if table.get('type') == 'table' and table.get('structured_data'):
            sample_table = table
            break
    if sample_table:
        break

if sample_table:
    print(f"Description: {sample_table['description']}")
    print(f"Confidence: {sample_table['confidence']}")
    print(f"Structured Data:")
    print(sample_table['structured_data'])
else:
    print("No tables with structured data found.")


📋 Sample Table Data:

No tables with structured data found.


In [18]:
# Step 5: Load the Markdown report
markdown_file = os.path.join(output_dir, "exports", "2025q1-alphabet-earnings-release_report.md")

print(f"📝 Loading Markdown report from: {markdown_file}")

with open(markdown_file, 'r', encoding='utf-8') as f:
    markdown_content = f.read()

print(f"✅ Markdown report loaded!")
print(f"📄 Length: {len(markdown_content):,} characters")
print(f"📊 Lines: {len(markdown_content.splitlines())}")

# Count tables and figures in markdown
table_count = markdown_content.count('#### Table')
figure_count = markdown_content.count('#### Figure')

print(f"\n📊 Markdown Content:")
print(f"   Tables: {table_count}")
print(f"   Figures: {figure_count}")
print(f"   Total Elements: {table_count + figure_count}")


📝 Loading Markdown report from: ../demo_output/2025q1-alphabet-earnings-release/exports/2025q1-alphabet-earnings-release_report.md
✅ Markdown report loaded!
📄 Length: 23,541 characters
📊 Lines: 471

📊 Markdown Content:
   Tables: 0
   Figures: 0
   Total Elements: 0


In [19]:
# Step 6: Show a preview of the Markdown content
print("📖 Markdown Preview (first 1000 characters):\n")
print("=" * 50)
print(markdown_content[:1000])
print("=" * 50)
print("... (truncated)")


📖 Markdown Preview (first 1000 characters):

# 2025Q1 Alphabet Earnings Release

**Total Pages:** 10

---

## Page 2

### Text Content

Alphabet Announces First Quarter  2025  Results
MOUNTAIN VIEW, Calif. – April 24, 2025  – Alphabet Inc. (NASDAQ: GOOG, GOOGL) today announced financial 
results for the quarter ended  March 31, 2025 .
•Consolidated  Alphabet revenues in Q1 2025 increased 12%, or 14% in constant currency, year over year to 
$90.2 billion  reflecting robust momentum across the business, with Google Search & other, YouTube ads, 
Google subscriptions, platforms, and devices, and Google Cloud each delivering double -digit growth rates. 
•Google Services revenues increased 10% to $77.3 billion , reflecting strong performance across Google 
Search & other, Google subscriptions, platforms, and devices , and YouTube ads .
•Google Cloud revenues increased 28% to $12.3 billion , led by growth in Google Cloud Platform (GCP) 
across core GCP products, AI Infrastructure , and Genera

## Summary

This demo shows how to:

1. **Run the extraction pipeline** - Extract text, tables, and figures from PDFs
2. **Load JSON data** - Access structured data for programmatic use
3. **Explore extracted content** - See what was found on each page
4. **View table data** - Access structured financial data
5. **Load Markdown reports** - Get human-readable summaries

### Key Features:
- ✅ **Text extraction** from all pages
- ✅ **Table detection** with structured data
- ✅ **Figure detection** with descriptions
- ✅ **Confidence scores** for quality assessment
- ✅ **Multiple output formats** (JSON + Markdown)

### Perfect for RAG Systems:
- **Structured data** for precise queries
- **Text content** for semantic search
- **Source attribution** with page numbers
- **Quality metrics** with confidence scores


## ⚡ Performance Benefits of Concurrent Processing

The concurrent VLM processing provides significant speed improvements:

### Before (Sequential):
- 10 pages × 3 seconds each = 30 seconds
- One page at a time

### After (Concurrent with 5 workers):
- 10 pages ÷ 5 workers × 3 seconds = 6 seconds
- **5x faster processing!**

### Configuration Options:
- `max_workers=3`: Conservative (good for rate limits)
- `max_workers=5`: Balanced (default)
- `max_workers=10`: Aggressive (for high-performance systems)

### Rate Limiting:
OpenAI has rate limits, so adjust `max_workers` based on your API tier:
- Free tier: 1-2 workers
- Pay-as-you-go: 3-5 workers  
- Enterprise: 10+ workers
